# Epidemiology model

https://nbviewer.jupyter.org/github/pyro-ppl/pyro/blob/sir-tutorial-ii/tutorial/source/epi_regional.ipynb?fbclid=IwAR3Gv8tLuiEjOmZh7-NQUa_ggm_QUqtSc5TxRZ0_pSxVA7Y3lWWzSFGKjrA 


In [ ]:
!git clone https://github.com/pyro-ppl/pyro.git

fatal: destination path 'pyro' already exists and is not an empty directory.


In [ ]:
%cd /content/pyro


/content/pyro


In [ ]:
!pip install .[extras]

Processing /content/pyro
  Created wheel for pyro-ppl: filename=pyro_ppl-1.6.0+c340831b-cp37-none-any.whl size=657943 sha256=d9082525b2e2c070c2437bcd5581d12dba5dadb74811b959a37b813622d4fe8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ba3b3xge/wheels/7f/c8/0f/f4e71d6e55e68c8a64c8382f3f6eb829e6aa18323499202e37
Successfully built pyro-ppl
  Found existing installation: pyro-ppl 1.6.0+c340831b
    Uninstalling pyro-ppl-1.6.0+c340831b:
      Successfully uninstalled pyro-ppl-1.6.0+c340831b


In [ ]:
import os
import logging
import urllib.request
from collections import OrderedDict

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import pyro
import pyro.distributions as dist
from pyro.contrib.epidemiology import CompartmentalModel, binomial_dist, infection_dist
from pyro.ops.tensor_utils import convolve

%matplotlib inline
pyro.enable_validation(True)           
torch.set_default_dtype(torch.double) 


  ## Model without Policies
  

In [ ]:
class CovidModel(CompartmentalModel):
    def __init__(self, population, new_cases, new_recovered, new_deaths):
        '''
        population (int) – Total population = S + E + I + R.
        '''
        assert len(new_cases) == len(new_recovered) == len(new_deaths)

        compartments = ("S", "E", "I", "D")  # R is implicit.
        duration = len(new_cases)
        super().__init__(compartments, duration, population)

        self.new_cases = new_cases
        self.new_deaths = new_deaths
        self.new_recovered = new_recovered
        

    def global_model(self):
        tau_i = pyro.sample("rec_time", dist.Normal(15.0, 3.0))
        tau_e = pyro.sample("incub_time", dist.Normal(5.0, 1.0))
        # R0 = pyro.sample("R0", dist.LogNormal(0., 1.))
        R0 = pyro.sample("R0", dist.Normal(2.5, 0.5))
        rho = pyro.sample("rho", dist.Beta(10, 10))  # About 50% response rate.
        mort_rate = pyro.sample("mort_rate", dist.Beta(2, 50))  # About 2% mortality rate.
        rec_rate = pyro.sample("rec_rate",dist.Beta(10, 10)) # About 50% recovery rate.
        return R0, tau_e, tau_i, rho, mort_rate, rec_rate

    def initialize(self, params):
        # Start with a single infection.
        return {"S": self.population - 1, "E": 0, "I": 1, "D": 0}

    def transition(self, params, state, t):
        R0, tau_e, tau_i, rho, mort_rate, rec_rate = params

        # Sample flows between compartments.
        S2E = pyro.sample("S2E_{}".format(t),
                            infection_dist(individual_rate=R0 / tau_i,
                                          num_susceptible=state["S"],
                                          num_infectious=state["I"],
                                          population=self.population))
        E2I = pyro.sample("E2I_{}".format(t),
                            binomial_dist(state["E"], 1 / tau_e )) 
        I2R = pyro.sample("I2R_{}".format(t),
                            binomial_dist(state["I"], 1 / tau_i))
        I2D = pyro.sample("I2D_{}".format(t),
                          binomial_dist(state["I"], mort_rate / tau_i))

        # Update compartments with flows.
        state["S"] = state["S"] - S2E 
        state["E"] = state["E"] + S2E - E2I
        state["I"] = state["I"] + E2I - I2R - I2D
        state["D"] = state["D"] + I2D

        # Condition on observations.
        t_is_observed = isinstance(t, slice) or t < self.duration
        pyro.sample("new_cases_{}".format(t),
                    binomial_dist(S2E, rho),
                    obs=self.new_cases[t] if t_is_observed else None)
        pyro.sample("new_deaths_{}".format(t),
                      binomial_dist(I2D, 1),
                      obs=self.new_deaths[t] if t_is_observed else None)
        pyro.sample("new_recovered_{}".format(t),
                    binomial_dist(I2R, rho),
                    obs=self.new_recovered[t] if t_is_observed else None)
        
    def compute_flows(self, prev, curr, t):
        S2E = prev["S"] - curr["S"]  # S can only go to E.
        I2D = curr["D"] - prev["D"]  # D can only have come from I.
        # We deduce the remaining flows by conservation of mass:
        #   curr - prev = inflows - outflows
        E2I = prev["E"] - curr["E"] + S2E
        I2R = prev["I"] - curr["I"] + E2I - I2D
        return {
            "S2E_{}".format(t): S2E,
            "E2I_{}".format(t): E2I,
            "I2D_{}".format(t): I2D,
            "I2R_{}".format(t): I2R,
        }

## Create Country

In [ ]:
# function to make the time series of confirmed and daily confirmed cases for a specific country
def create_country (country, start_date, end_date, state = False) : 

    url = 'https://raw.githubusercontent.com/assemzh/ProbProg-COVID-19/master/full_grouped.csv'
    data = pd.read_csv(url)

    data.Date = pd.to_datetime(data.Date)

    if state :
        df = data.loc[data["Province/State"] == country, ["Province/State", "Date", "Confirmed", "Deaths", "Recovered", "Active", "New cases", "New deaths", "New recovered"]]
    else : 
        df = data.loc[data["Country/Region"] == country, ["Country/Region", "Date", "Confirmed", "Deaths", "Recovered", "Active", "New cases", "New deaths", "New recovered"]]
    df.columns = ["country", "date", "confirmed", "deaths", "recovered", "active", "new_cases", "new_deaths", "new_recovered"]

    # group by country and date
    df = df.groupby(['country','date'])['confirmed', 'deaths', 'recovered',"active", "new_cases", "new_deaths", "new_recovered"].sum().reset_index()

    # convert date string to datetime
    df.date = pd.to_datetime(df.date)
    df = df.sort_values(by = "date")
    df = df[df.date >= start_date]
    df = df[df.date <= end_date]

    active = df['active'].tolist()
    recovered = df['recovered'].tolist()
    deaths = df['deaths'].tolist()
    new_cases = df['new_cases'].tolist()
    new_recovered = df['new_recovered'].tolist()
    new_deaths = df['new_deaths'].tolist()
    
    active = torch.tensor(list(map(float, active))).view(len(active),1) 
    recovered = torch.tensor(list(map(float, recovered))).view(len(recovered),1) 
    deaths = torch.tensor(list(map(float, deaths))).view(len(deaths),1) 
    new_cases = torch.tensor(list(map(float, new_cases))).view(len(new_cases),1)  
    new_recovered = torch.tensor(list(map(float, new_recovered))).view(len(new_recovered),1) 
    new_deaths = torch.tensor(list(map(float, new_deaths))).view(len(new_deaths),1) 


    return_data = {
        'active':active,
        'recovered':recovered,
        'deaths':deaths,
        'new_cases':new_cases,
        'new_recovered': new_recovered,
        'new_deaths':new_deaths }
        
    return return_data


## Get data for countries


In [ ]:
Japan = create_country("Japan", start_date = "2020-02-01", end_date = "2020-04-01")
Sweden = create_country("Sweden", start_date = "2020-02-01", end_date = "2020-04-01")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  app.launch_new_instance()


##Train the model using MCMC.



In [ ]:
%%time
Sweden_model = CovidModel(10230000, Sweden["new_cases"], Sweden["new_recovered"], Sweden["new_deaths"] )
pyro.set_rng_seed(20210521)
Sweden_mcmc = Sweden_model.fit_mcmc(num_samples=500, warmup_steps = 200)
Sweden_mcmc.summary()

INFO 	 Running inference...
Sample: 100%|██████████| 700/700 [00:49, 14.03it/s, step size=7.64e-05, acc. prob=0.720]



                     mean       std    median      5.0%     95.0%     n_eff     r_hat
             R0      2.77      0.00      2.77      2.77      2.77     12.46      1.01
 auxiliary[0,0] 10229991.22      0.00 10229991.22 10229991.22 10229991.22      5.34      1.41
 auxiliary[0,1] 10229983.34      0.00 10229983.34 10229983.34 10229983.34      3.00      2.03
 auxiliary[0,2] 10229974.97      0.00 10229974.97 10229974.97 10229974.97      2.80      2.54
 auxiliary[0,3] 10229966.19      0.00 10229966.19 10229966.19 10229966.19      7.19      1.33
 auxiliary[0,4] 10229957.06      0.00 10229957.06 10229957.06 10229957.07      2.59      2.66
 auxiliary[0,5] 10229947.77      0.00 10229947.77 10229947.77 10229947.77      9.10      1.09
 auxiliary[0,6] 10229938.37      0.00 10229938.37 10229938.36 10229938.37      2.74      2.15
 auxiliary[0,7] 10229928.89      0.00 10229928.89 10229928.89 10229928.89      3.56      1.55
 auxiliary[0,8] 10229950.86      0.00 10229950.86 10229950.86 10229950.86  

In [ ]:
%%time
Sweden_model = CovidModel(10230000, Sweden["new_cases"], Sweden["new_recovered"], Sweden["new_deaths"] )
Sweden_mcmc = Sweden_model.fit_mcmc(num_samples=500, warmup_steps = 200)
Sweden_mcmc.summary()

INFO 	 Running inference...
Sample: 100%|██████████| 700/700 [00:25, 27.48it/s, step size=1.86e-04, acc. prob=0.370]



                     mean       std    median      5.0%     95.0%     n_eff     r_hat
             R0      2.67      0.00      2.67      2.67      2.67      6.16      1.34
 auxiliary[0,0] 10229990.38      0.00 10229990.38 10229990.38 10229990.38     16.52      1.13
 auxiliary[0,1] 10229981.77      0.00 10229981.77 10229981.77 10229981.77      6.82      1.67
 auxiliary[0,2] 10229973.01      0.00 10229973.01 10229973.00 10229973.01      2.63      2.26
 auxiliary[0,3] 10229964.13      0.00 10229964.13 10229964.13 10229964.13      8.23      1.18
 auxiliary[0,4] 10229955.09      0.00 10229955.09 10229955.09 10229955.09      7.89      1.12
 auxiliary[0,5] 10229945.92      0.00 10229945.92 10229945.92 10229945.92      5.02      1.57
 auxiliary[0,6] 10229936.61      0.00 10229936.61 10229936.61 10229936.62      2.63      2.49
 auxiliary[0,7] 10229927.17      0.00 10229927.17 10229927.16 10229927.17      2.46      2.83
 auxiliary[0,8] 10229917.56      0.00 10229917.56 10229917.55 10229917.56  

In [ ]:
%%time
Sweden_model = CovidModel(10230000, Sweden["new_cases"], Sweden["new_recovered"], Sweden["new_deaths"] )
Sweden_mcmc = Sweden_model.fit_mcmc(num_samples=500, warmup_steps = 200)
Sweden_mcmc.summary()

INFO 	 Running inference...
Sample: 100%|██████████| 700/700 [01:29,  7.83it/s, step size=9.38e-04, acc. prob=0.895]



                     mean       std    median      5.0%     95.0%     n_eff     r_hat
             R0      2.93      0.00      2.93      2.93      2.93      3.30      1.68
 auxiliary[0,0] 10229990.62      0.23 10229990.69 10229990.18 10229990.86      3.31      1.55
 auxiliary[0,1] 10229982.16      0.80 10229982.45 10229980.74 10229982.95      2.66      2.32
 auxiliary[0,2] 10229973.08      1.87 10229973.41 10229970.00 10229975.43      2.54      2.59
 auxiliary[0,3] 10229962.96      3.45 10229963.32 10229957.16 10229967.39      2.51      2.67
 auxiliary[0,4] 10229951.60      5.55 10229951.96 10229942.53 10229959.03      2.50      2.69
 auxiliary[0,5] 10229938.92      8.15 10229939.24 10229925.82 10229950.08      2.50      2.70
 auxiliary[0,6] 10229924.89     11.19 10229925.16 10229907.12 10229940.45      2.50      2.70
 auxiliary[0,7] 10229909.57     14.63 10229909.70 10229886.55 10229930.14      2.50      2.70
 auxiliary[0,8] 10229893.01     18.42 10229893.00 10229864.18 10229919.10  

In [ ]:
%%time
Sweden_model = CovidModel(10230000, Sweden["new_cases"], Sweden["new_recovered"], Sweden["new_deaths"] )
Sweden_mcmc = Sweden_model.fit_mcmc(num_samples=500, warmup_steps = 200)
Sweden_mcmc.summary()

INFO 	 Running inference...
Sample: 100%|██████████| 700/700 [01:51,  6.29it/s, step size=4.76e-04, acc. prob=0.779]



                     mean       std    median      5.0%     95.0%     n_eff     r_hat
             R0      2.38      0.00      2.38      2.38      2.38      3.33      1.67
 auxiliary[0,0] 10229989.96      0.14 10229990.00 10229989.74 10229990.14      2.72      2.21
 auxiliary[0,1] 10229979.62      0.79 10229979.83 10229978.39 10229980.67      2.64      2.33
 auxiliary[0,2] 10229967.07      2.25 10229967.56 10229963.65 10229970.21      2.61      2.39
 auxiliary[0,3] 10229951.98      4.50 10229952.82 10229945.21 10229958.62      2.59      2.42
 auxiliary[0,4] 10229934.37      7.51 10229935.62 10229923.24 10229945.76      2.59      2.43
 auxiliary[0,5] 10229914.37     11.10 10229916.05 10229898.05 10229931.60      2.59      2.44
 auxiliary[0,6] 10229892.23     15.11 10229894.30 10229870.20 10229916.25      2.59      2.44
 auxiliary[0,7] 10229868.19     19.43 10229870.60 10229838.49 10229897.88      2.59      2.45
 auxiliary[0,8] 10229842.47     23.94 10229845.08 10229806.09 10229879.55  

In [ ]:
%%time
Sweden_model = CovidModel(10230000, Sweden["new_cases"], Sweden["new_recovered"], Sweden["new_deaths"] )
Sweden_mcmc = Sweden_model.fit_mcmc(num_samples=500, warmup_steps = 200)
Sweden_mcmc.summary()

INFO 	 Running inference...
Sample: 100%|██████████| 700/700 [01:20,  8.70it/s, step size=1.74e-03, acc. prob=0.597]



                     mean       std    median      5.0%     95.0%     n_eff     r_hat
             R0      2.49      0.00      2.49      2.49      2.49      6.33      1.32
 auxiliary[0,0] 10229991.15      0.59 10229991.11 10229990.07 10229991.94      2.73      2.25
 auxiliary[0,1] 10229981.86      1.44 10229981.75 10229979.25 10229983.84      2.73      2.26
 auxiliary[0,2] 10229971.31      2.52 10229971.10 10229966.86 10229974.77      2.71      2.27
 auxiliary[0,3] 10229959.59      3.78 10229959.26 10229952.94 10229964.80      2.72      2.26
 auxiliary[0,4] 10229946.74      5.22 10229946.28 10229937.59 10229953.90      2.72      2.26
 auxiliary[0,5] 10229932.85      6.82 10229932.21 10229920.95 10229942.20      2.73      2.25
 auxiliary[0,6] 10229917.93      8.56 10229917.14 10229903.07 10229929.66      2.73      2.25
 auxiliary[0,7] 10229902.02     10.44 10229901.01 10229883.94 10229916.37      2.73      2.25
 auxiliary[0,8] 10229885.12     12.44 10229883.88 10229863.63 10229902.22  

In [ ]:
%%time
Sweden_model = CovidModel(10230000, Sweden["new_cases"], Sweden["new_recovered"], Sweden["new_deaths"] )
Sweden_mcmc = Sweden_model.fit_mcmc(num_samples=500, warmup_steps = 200)
Sweden_mcmc.summary()

INFO 	 Running inference...
Sample: 100%|██████████| 700/700 [02:08,  5.44it/s, step size=7.26e-04, acc. prob=0.898]



                     mean       std    median      5.0%     95.0%     n_eff     r_hat
             R0      2.82      0.00      2.83      2.82      2.83      5.96      1.04
 auxiliary[0,0] 10229991.30      0.24 10229991.31 10229990.95 10229991.65      2.46      2.73
 auxiliary[0,1] 10229982.96      0.50 10229982.92 10229982.26 10229983.75      2.50      2.62
 auxiliary[0,2] 10229974.33      0.69 10229974.30 10229973.38 10229975.40      2.48      2.66
 auxiliary[0,3] 10229965.47      0.82 10229965.45 10229964.33 10229966.73      2.46      2.71
 auxiliary[0,4] 10229956.42      0.93 10229956.42 10229955.09 10229957.80      2.46      2.71
 auxiliary[0,5] 10229947.20      1.03 10229947.22 10229945.74 10229948.71      2.47      2.70
 auxiliary[0,6] 10229969.94      1.89 10229970.59 10229966.75 10229971.87      2.66      2.27
 auxiliary[0,7] 10229959.88      2.38 10229960.50 10229956.00 10229962.59      2.59      2.41
 auxiliary[0,8] 10229949.13      3.24 10229949.72 10229944.03 10229953.13  

In [ ]:
%%time
Sweden_model = CovidModel(10230000, Sweden["new_cases"], Sweden["new_recovered"], Sweden["new_deaths"] )
Sweden_mcmc = Sweden_model.fit_mcmc(num_samples=500, warmup_steps = 200)
Sweden_mcmc.summary()

INFO 	 Running inference...
Sample: 100%|██████████| 700/700 [01:53,  6.16it/s, step size=9.15e-04, acc. prob=0.850]



                     mean       std    median      5.0%     95.0%     n_eff     r_hat
             R0      2.52      0.00      2.52      2.52      2.52      4.59      1.25
 auxiliary[0,0] 10229990.17      0.13 10229990.20 10229989.96 10229990.34      2.46      2.72
 auxiliary[0,1] 10229981.13      0.31 10229981.15 10229980.68 10229981.57      2.47      2.73
 auxiliary[0,2] 10229971.85      0.49 10229971.87 10229971.15 10229972.55      2.47      2.71
 auxiliary[0,3] 10229962.41      0.67 10229962.44 10229961.42 10229963.35      2.47      2.71
 auxiliary[0,4] 10229952.82      0.87 10229952.93 10229951.53 10229954.01      2.46      2.75
 auxiliary[0,5] 10229943.01      1.07 10229943.16 10229941.44 10229944.48      2.45      2.77
 auxiliary[0,6] 10229951.64      6.38 10229953.10 10229941.04 10229959.63      2.48      2.67
 auxiliary[0,7] 10229940.87      6.80 10229942.33 10229929.72 10229949.52      2.47      2.69
 auxiliary[0,8] 10229929.20      7.45 10229930.61 10229917.20 10229938.86  